In [1]:
import pandas as pd
from konlpy.tag import Mecab
import pickle

In [3]:
# tagger = Mecab()

In [3]:
with open('../file/file_pos.p', 'rb') as f:
    file_pos = pickle.load(f)

In [33]:
file_pos['pos_splitted'] = file_pos.article_original_pos.apply(lambda e: [list(list(zip(*ee))[0]) for ee in e])

In [27]:
file_pos['abstractive_splitted'] = file_pos.abstractive.apply(lambda e: [_[0] for _ in tagger.pos(e)])

In [34]:
file_pos.pos_splitted

0        [[당진시, 문화, 관광, 과, 를, 대상, 으로, 하, 는, 행정, 사무, 감사,...
1        [[미국, 메이저, 리그, (, MLB, ), 에서, 활동, 하, 는, 한국, 선수...
2        [[인천, 부영, 공원, 운영, 생활, 야구, 협회, 80, 여, 동호회, 팀, 에...
3        [[대구, ·, 경북, 첨단, 의료, 산업, 진흥, 재단, 의약, 생산, 센터, 는...
4        [[국내, 유통, 되, 는, 탄산, 음료, 중, 식품, 의, 약, 품안, 전처, 에...
                               ...                        
42798    [[칠, 곡, 북삼, 초등, 학교, (, 교장, 서금, 자, ), 에서, 는, 지난...
42799    [[김상훈, 의원], [최근, 5, 년, 간, ,, LH, 공공, 임대, 임차인, ...
42800    [[▲, 5, 일, 오전, 파주, 임진, 각, 에서, 열린, ‘, DMZ, 155,...
42801    [[당진, 지역, 휘발유, 평균, 가격, 은, 1534, 원, ,, 경유, 평균, ...
42802    [[양승조, 충남, 도, 지사, 는, 12, 일, 대기, 오염, 물질, 배출, 주요...
Name: pos_splitted, Length: 42803, dtype: object

In [39]:
file_pos['extractive_tokens'] = file_pos.apply(lambda e: [word for idx in e.extractive 
                                               for word in e.pos_splitted[idx]], axis=1)

In [44]:
file_pos['rouge-1'] = file_pos.apply(lambda e: len(set(e.abstractive_splitted).intersection(e.extractive_tokens)),
                                     axis=1)

In [45]:
file_pos['rouge-1']

0        38
1        45
2        65
3        52
4        56
         ..
42798    43
42799    43
42800    32
42801    37
42802    40
Name: rouge-1, Length: 42803, dtype: int64

In [51]:
file_pos['rouge-1_each_sen'] = file_pos.apply(lambda e: [len(set(sen).intersection(e.extractive_tokens))
                          for sen in e.pos_splitted], axis=1)

In [62]:
file_pos['rouge-1_each_sen_sorted'] = file_pos['rouge-1_each_sen'].apply(lambda e: sorted(enumerate(e), 
                                                                                          key=lambda e: -e[1])[:3])

In [63]:
file_pos[['extractive', 'rouge-1_each_sen_sorted']]

,extractive,rouge-1_each_sen_sorted
0,"[0, 1, 2]","[(1, 67), (2, 57), (0, 36)]"
1,"[0, 1, 3]","[(1, 36), (0, 23), (2, 18)]"
2,"[6, 7, 10]","[(6, 48), (16, 31), (10, 30)]"
3,"[1, 6, 4]","[(1, 38), (6, 26), (5, 24)]"
4,"[2, 3, 10]","[(2, 45), (10, 41), (3, 30)]"
...,...,...
42798,"[0, 1, 2]","[(0, 40), (1, 36), (2, 35)]"
42799,"[1, 2, 4]","[(2, 70), (4, 37), (1, 31)]"
42800,"[2, 3, 6]","[(3, 39), (6, 37), (2, 31)]"
42801,"[4, 6, 0]","[(0, 37), (4, 25), (6, 23)]"


In [66]:
file_pos['selected_by_abstractive'] = file_pos['rouge-1_each_sen_sorted'].apply(lambda e: [_[0] for _ in e])

In [73]:
from collections import Counter

In [74]:
Counter(file_pos.apply(lambda e: len(set(e.extractive).intersection(e.selected_by_abstractive)), axis=1))

Counter({3: 29712, 2: 12779, 1: 312})

In [81]:
file_pos['sentences_by_abstractive'] = file_pos.apply(lambda e: [word
                                                                 for _ in e.selected_by_abstractive
                                                                 for word in e.pos_splitted[_]], axis=1)

In [84]:
file_pos['rouge-1_by_vote'] = file_pos.apply(lambda e: 
                                             len(set(e.abstractive_splitted).intersection(e.sentences_by_abstractive)),
                                             axis=1)

In [85]:
file_pos[['rouge-1', 'rouge-1_by_vote']]

,rouge-1,rouge-1_by_vote
0,38,38
1,45,41
2,65,62
3,52,44
4,56,56
...,...,...
42798,43,43
42799,43,43
42800,32,32
42801,37,37


In [92]:
file_pos[file_pos['rouge-1'] >= file_pos['rouge-1_by_vote']].shape

(41294, 16)

In [91]:
file_pos[file_pos['rouge-1'] < file_pos['rouge-1_by_vote']].shape

(1509, 16)